<a href="https://colab.research.google.com/github/9490549112/22A81A0574-/blob/main/DA_day7_neural_networks_brain_tumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#Define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [3]:
#define data generator for train,validation and test sets
train_datagen=ImageDataGenerator(
    rescale=1./225,
    validation_split=0.2
)
train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
target_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE,
class_mode='binary',
subset='training'
)
val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',target_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE,
class_mode='binary',
subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
target_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE,
class_mode='binary'
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 70 images belonging to 1 classes.


In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
])

In [5]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [6]:
history=model.fit(train_generator,validation_data=val_generator,epochs=2)

Epoch 1/2
75/75 [==============================] - 686s 9s/step - loss: 0.4749 - accuracy: 0.7917 - val_loss: 0.2728 - val_accuracy: 0.8800
Epoch 2/2
75/75 [==============================] - 330s 4s/step - loss: 0.2065 - accuracy: 0.9237 - val_loss: 0.1712 - val_accuracy: 0.9433


In [7]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
#load the saved model
model=load_model('/content/Model.h5')
#load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test/pred/pred1.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0) #add batch dimension
img_array /=225. #normalize the pixel values
#make predictions
prediction=model.predict(img_array)
#print the prediction
if prediction <0.5:
  print("prediction: no tumor(probability:",prediction[0][0],")")
else:
  print("prediction: tumor present(probability:",prediction[0][0],")")

1/1 [==============================] - 0s 182ms/step
prediction: no tumor(probability: 0.021330273 )
